In [5]:
#get proper modules
import pandas as pd
import numpy as np
import sklearn as sk

#load data (all feature should be combined to one dframe here)
data = pd.read_csv("datasets/amalgamated_data.csv")

#format state names properly
for index, row in data.iterrows():
    state = (row['state'])
    state = state[1:len(state)-1] 
    data.set_value(index, 'state', state)

#show head to ensure proper import
data.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


,Unnamed: 0,latitude,longitude,barometric pressure noons:sea level pressure,barometric pressure noons:pressure,month,year,state,humidities:relative humidity (%),humidities:dewpoint,air_temp
0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9
1,725218,39.724047,-77.523373,980.1,952.1,11,2017,Pennsylvania,59.0,-1.2,-1.3
2,8176112,34.151827,-118.130506,1014.9,1012.2,2,2016,California,68.0,11.2,13.1
3,4143373,43.700000,-72.100000,1000.1,997.2,10,2014,New Hampshire,83.0,-2.9,4.1
4,7782185,41.810378,-71.449503,997.5,994.6,12,2017,Rhode Island,55.0,-20.4,9.1


In [1]:
#get proper modules
import pandas as pd
import numpy as np
import sklearn as sk

health = pd.read_csv("datasets/health_data.csv")

#START RUN HERE TO AVOID HAVING TO REMANIPULATING LARGE DATASET *UNCOMMENT BELOW*
data = pd.read_csv("datasets/amalgamated_data.csv")

#merge data and health
data = data.merge(health, left_on=['state','month','year'], right_on=['State','month','Year'])


In [2]:
data.head()

,Unnamed: 0,Unnamed: 0.1,latitude,longitude,barometric pressure noons:sea level pressure,barometric pressure noons:pressure,month,year,state,humidities:relative humidity (%),humidities:dewpoint,air_temp,index,State,Year,Infected
0,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19386,Pennsylvania,2017,60038
1,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19439,Pennsylvania,2017,62684
2,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19492,Pennsylvania,2017,63190
3,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19545,Pennsylvania,2017,55326
4,1,725218,39.724047,-77.523373,980.1,952.1,11,2017,Pennsylvania,59.0,-1.2,-1.3,19386,Pennsylvania,2017,60038


In [3]:
data.drop(data.columns[[0,1,2,3,6,7,8,12,13,14]], 1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000611 entries, 0 to 2000610
Data columns (total 6 columns):
barometric pressure noons:sea level pressure    float64
barometric pressure noons:pressure              float64
humidities:relative humidity (%)                float64
humidities:dewpoint                             float64
air_temp                                        float64
Infected                                        int64
dtypes: float64(5), int64(1)
memory usage: 106.8 MB


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

def split_train_set(data, test_ratio):
    shuf_index = np.random.permutation(len(data))
    test_size = int(len(data) * test_ratio)
    test_indexes = shuf_index[:test_size]
    train_indexes = shuf_index[test_size:]
    return data.iloc[train_indexes], data.iloc[test_indexes]

#handle dataframe and numpy array transformations
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def _init_(self, attribute_names):
        self.atrribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

#drop nulls (failsafe)
data.dropna()

#spilt training and test set (USE TRAIN FOR ALL REGRESSORS)
train_data, test_data = split_train_set(data, .2)

#seperate labels and attributes 
train_data_labels = train_data['Infected']
train_data.drop(train_data.columns[[5]], 1, inplace=True)
#train_data.drop(['state'], 1, inplace = True)

test_data_labels = test_data['Infected']
test_data.drop(test_data.columns[[5]], 1, inplace=True)
#test_data.drop(['state'], 1, inplace = True)

train_data.info()

#add additional transformation tasks to the below sklearn pipeline
#standardize data (use devide by variance standardization to minimize outlier impact)
stpipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
train_data_prepped = stpipeline.fit_transform(train_data)
test_data_prepped = stpipeline.fit_transform(test_data)



/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600489 entries, 1497119 to 1547265
Data columns (total 5 columns):
barometric pressure noons:sea level pressure    1600489 non-null float64
barometric pressure noons:pressure              1600489 non-null float64
humidities:relative humidity (%)                1600489 non-null float64
humidities:dewpoint                             1600489 non-null float64
air_temp                                        1600489 non-null float64
dtypes: float64(5)
memory usage: 73.3 MB


In [5]:
print(train_data_prepped)
print(train_data_labels.head())

[[-1.12674921 -1.05816193 -1.90758534  0.66823403  1.72526307]
 [ 1.06975233  1.15047839  1.00866906  0.0287552   0.68489526]
 [ 0.68536456  0.90364921 -0.63172404 -2.24385421 -2.32290079]
 ...
 [-1.01692413 -0.99084488  0.64413726 -0.19752193 -0.77780008]
 [ 1.36927527  1.34922397  1.31244556  0.86499675  1.37504024]
 [-1.35139141 -1.19920717 -0.57096874  0.21567978  1.00421607]]
1497119    57073
328377      4972
792173      5943
1539858    63334
827021      5943
Name: Infected, dtype: int64


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Score: ")
    print(scores)
    print("\nMean: ")
    print(scores.mean())
    print("\nStd. Dev.: ") 
    print(scores.std())
    
lin_reg = LinearRegression() 
lin_reg.fit(train_data_prepped, train_data_labels)

lin_scores = cross_val_score(lin_reg, train_data_prepped, train_data_labels, 
    scoring = "neg_mean_squared_error", cv=10) 
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


Score: 
[15646.25724648 15662.34611482 15695.68482638 15668.88851334
 15660.74739276 15669.89396034 15706.09664812 15714.40526022
 15678.46177477 15665.80691477]

Mean: 
15676.85886520016

Std. Dev.: 
20.64559617759353


In [7]:
print(lin_reg.coef_)

[ 16061.62528373 -36809.9724762   -1323.68087222  -1185.56310396
  -1425.4322945 ]


In [ ]:
forest_reg = RandomForestRegressor() 
forest_reg.fit(train_data_prepped, train_data_labels)

forest_scores = cross_val_score(forest_reg, train_data_prepped, train_data_labels, 
    scoring = "neg_mean_squared_error", cv=10) 
forest_rmse_scores = np.sqrt(-forest_scores)

display_scores(forest_rmse_scores)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
from sklearn.metrics import mean_squared_error
lin_prediction = lin_reg.predict(test_data_prepped)
mse = mean_squared_error(test_data_labels, lin_prediction)
rmse = np.sqrt(mse)

print(rmse)

15705.601618086675
